In [27]:
import os
import time as tm
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sa
from dotenv import load_dotenv
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [7]:
load_dotenv()

# Load the model
model = tf.keras.models.load_model('../../models/F1.model.h5')

In [47]:
# LOAD DATA 
date_now = dt.datetime(2024, 1, 1)
# date_3_days_back = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
date_3_days_back = (date_now - dt.timedelta(days=3)).strftime('%Y-%m-%d')
STOCK = 'F'
DB_URI = f'postgresql://{os.environ["PG_USER"]}:{os.environ["PG_PASS"]}@{os.environ["PG_URL"]}/{os.environ["PG_DB"]}'
QUERY = f"SELECT * FROM stocks_daily_price WHERE symbol = '{STOCK}' AND date >= '{date_3_days_back}'"
# coltype = (
#     sa.Column('symbol', sa.String),
#     # sa.Column('date', sa.Date, primary_key=True),
#     sa.Column('open', sa.Float),
#     sa.Column('high', sa.Float),
#     sa.Column('low', sa.Float),
#     sa.Column('close', sa.Float),
#     sa.Column('volume', sa.Integer),
# )
try:
    engine = sa.create_engine(DB_URI)
    with engine.connect() as connection:
        init_df = pd.read_sql_query(QUERY,
                                    index_col=['date'],
                                    parse_dates=['date'],
                                    # dtype=coltype,
                                    con=connection)
        print(init_df.shape)
        print(init_df.info())
        print(init_df.head())
except Exception as e:
    print(e)
finally:
    if 'engine' in locals():
        engine.dispose()

(283, 6)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 283 entries, 2023-12-29 to 2025-02-14
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  283 non-null    object 
 1   volume  283 non-null    int64  
 2   close   283 non-null    float64
 3   open    283 non-null    float64
 4   high    283 non-null    float64
 5   low     283 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 15.5+ KB
None
           symbol    volume      close       open       high        low
date                                                                   
2023-12-29      F  40902400  11.392626  11.542161  11.579544  11.364588
2024-01-02      F  47494800  11.364590  11.252439  11.570199  11.158980
2024-01-03      F  70120100  10.944025  11.233748  11.243093  10.878604
2024-01-04      F  54920300  10.915987  10.990754  11.046829  10.869257
2024-01-05      F  44409300  11.074866  10.887948  11.252438  10.869257


In [48]:
# remove columns which our neural network will not use
init_df = init_df.drop(['open', 'high', 'low', 'symbol', 'volume'], axis=1)
# create the column 'date' based on index column
init_df['date'] = init_df.index

In [49]:
# Ensure the 'close' column is numeric
# init_df['close'] = pd.to_numeric(init_df['close'], errors='coerce')

# Drop any rows with NaN values that may have been introduced by the conversion
init_df = init_df.dropna(subset=['close'])


In [50]:
scaler = MinMaxScaler()
init_df['scaled_close'] = scaler.fit_transform(np.expand_dims(init_df['close'].values, axis=1))

In [51]:
init_df

,close,date,scaled_close
date,,,
2023-12-29,11.392626,2023-12-29,0.443191
2024-01-02,11.364590,2024-01-02,0.437499
2024-01-03,10.944025,2024-01-03,0.352101
2024-01-04,10.915987,2024-01-04,0.346408
2024-01-05,11.074866,2024-01-05,0.378669
...,...,...,...
2025-02-10,9.240000,2025-02-10,0.006092
2025-02-11,9.210000,2025-02-11,0.000000
2025-02-12,9.230000,2025-02-12,0.004061


In [52]:

# Predict the next 3 days prices
predictions = model.predict(init_df['scaled_close'])
# y_predicted_transformed = np.squeeze(scaler.inverse_transform(predictions))

# Print the predictions
print(predictions)
# print("Predicted prices for the next 3 days:", y_predicted_transformed)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.4366482 ]
 [0.4335006 ]
 [0.3860646 ]
 [0.38288832]
 [0.40086466]
 [0.4135208 ]
 [0.3998087 ]
 [0.3987527 ]
 [0.3860646 ]
 [0.35954916]
 [0.35954916]
 [0.33932948]
 [0.30943757]
 [0.3318666 ]
 [0.3318666 ]
 [0.34997833]
 [0.31371427]
 [0.34678525]
 [0.35210627]
 [0.36910683]
 [0.3934691 ]
 [0.3871228 ]
 [0.42719942]
 [0.431401  ]
 [0.3733505 ]
 [0.4240458 ]
 [0.50020033]
 [0.50330186]
 [0.48777008]
 [0.5187738 ]
 [0.48777008]
 [0.47530347]
 [0.5061766 ]
 [0.48277628]
 [0.47743988]
 [0.46567708]
 [0.46353513]
 [0.46567708]
 [0.44528866]
 [0.45066255]
 [0.48277628]
 [0.49768263]
 [0.49874544]
 [0.5294404 ]
 [0.5125352 ]
 [0.49130076]
 [0.4944929 ]
 [0.46995807]
 [0.4624638 ]
 [0.4624638 ]
 [0.49023604]
 [0.45924824]
 [0.4571033 ]
 [0.46995807]
 [0.48277628]
 [0.5462689 ]
 [0.54836696]
 [0.547318  ]
 [0.5462689 ]
 [0.49768263]
 [0.56301725]
 [0.58590984]
 [0.58694655]
 [0.58590984]
 [0.62403286]
 [0.5786435 ]
 [0.58590984]
 [0.59936   ]
 [0.612750

In [55]:
# Create a date range for the next 3 days
future_dates = pd.date_range(start=init_df.index[-1] + pd.Timedelta(days=1), periods=predictions.shape[0], freq='D')

# Inverse transform the predictions to get the actual prices
predicted_prices = scaler.inverse_transform(predictions)

# Create a DataFrame with the future dates and predicted prices
predictions_df = pd.DataFrame(data={'date': future_dates, 'predicted_close': predicted_prices.flatten()})

# Print the predictions DataFrame
print(predictions_df)

          date  predicted_close
0   2025-02-15        11.360401
1   2025-02-16        11.344900
2   2025-02-17        11.111287
3   2025-02-18        11.095645
4   2025-02-19        11.184175
..         ...              ...
278 2025-11-20        10.149955
279 2025-11-21        10.132904
280 2025-11-22        10.144271
281 2025-11-23        10.212457
282 2025-11-24        10.286278

[283 rows x 2 columns]


In [54]:
init_df

,close,date,scaled_close
date,,,
2023-12-29,11.392626,2023-12-29,0.443191
2024-01-02,11.364590,2024-01-02,0.437499
2024-01-03,10.944025,2024-01-03,0.352101
2024-01-04,10.915987,2024-01-04,0.346408
2024-01-05,11.074866,2024-01-05,0.378669
...,...,...,...
2025-02-10,9.240000,2025-02-10,0.006092
2025-02-11,9.210000,2025-02-11,0.000000
2025-02-12,9.230000,2025-02-12,0.004061
